In [41]:
from functional import seq

from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

import re

In [42]:
HEADERS = {'User-Agent': 'Mozilla/5.0'}

In [ ]:
def hymn_to_verses(hymn):
    english = (seq(hymn["english"].split("\n"))
                .map(lambda line: line.strip())
                .map(lambda line: re.sub(r"^[^a-zA-Z]+", "", line))
                .grouped(2))

In [43]:
def url_to_hymn(i, url):
    san_req = Request(url, headers=HEADERS)
    san_html = urlopen(san_req).read().decode('utf-8')

    eng_req = Request(url.replace('rvsan', 'rigveda'), headers=HEADERS)
    eng_html = urlopen(eng_req).read().decode('utf-8')
    eng_soup = BeautifulSoup(eng_html, 'html.parser')

    devanagari = re.findall(r'(?<=<\/H3>)(.*)(?=<P><\/P>)', san_html, flags=re.S)[0]
    iast = re.findall(r'(?=<P><\/P>)(.*)<P><HR>', san_html, flags=re.S)[0]
    english = eng_soup.find_all('p')[1].text.get_text(separator="\n").strip()

    devanagari, iast = seq(devanagari, iast) \
        .map(lambda text: re.sub(r" *<BR\>", "", text)) \
        .map(lambda text: text.strip())
    hymn = {'devanagari': devanagari, 'iast': iast, 'english': english}
    print("hymn", hymn)
    print("url", url)
    return {
        'url': url,
        'hymn': i + 1,
        'verses': [verse_to_sentences(verse) for verse in hymn_to_verses(hymn)]
    }

In [44]:
def url_to_book(i, url):
    req = Request(url, headers=HEADERS)
    book_html = urlopen(req).read()
    soup = BeautifulSoup(book_html, 'html.parser')

    return {
        'url': url,
        'book': i + 1,
        'hymns': [url_to_hymn(i, url) for i, url in enumerate(seq(soup.find_all('a'))
                                                              .map(lambda a: a.get('href'))
                                                              .filter(lambda href: href and href.startswith('rv'))
                                                              .take(3)
                                                              .map(lambda href: f"https://www.sacred-texts.com/hin/rvsan/{href}"))]
    }


In [45]:
book_urls = [f"https://www.sacred-texts.com/hin/rvsan/rvi{(i + 1):02}.htm" for i in range(10)]
books = [url_to_book(i, url) for i, url in enumerate(book_urls)]

In [46]:
books

hymn {'devanagari': 'अग्निमीळे पुरोहितं यज्ञस्य देवं रत्वीजम |\n होतारं रत्नधातमम ||\n अग्निः पूर्वेभिर्र्षिभिरीड्यो नूतनैरुत |\n स देवानेह वक्षति ||\n अग्निना रयिमश्नवत पोषमेव दिवे-दिवे |\n यशसं वीरवत्तमम ||\n अग्ने यं यज्ञमध्वरं विश्वतः परिभूरसि |\n स इद्देवेषु गछति ||\n अग्निर्होता कविक्रतुः सत्यश्चित्रश्रवस्तमः |\n देवो देवेभिरा गमत ||\n यदङग दाशुषे तवमग्ने भद्रं करिष्यसि |\n तवेत तत सत्यमङगिरः ||\n उप तवाग्ने दिवे-दिवे दोषावस्तर्धिया वयम |\n नमो भरन्त एमसि ||\n राजन्तमध्वराणां गोपां रतस्य दीदिविम |\n वर्धमानंस्वे दमे ||\n स नः पितेव सूनवे.अग्ने सूपायनो भव |\n सचस्वा नः सवस्तये ||', 'iast': '<P></P>\n aghnimīḷe purohitaṃ yajñasya devaṃ ṛtvījam |\n hotāraṃ ratnadhātamam ||\n aghniḥ pūrvebhirṛṣibhirīḍyo nūtanairuta |\n sa devāneha vakṣati ||\n aghninā rayimaśnavat poṣameva dive-dive |\n yaśasaṃ vīravattamam ||\n aghne yaṃ yajñamadhvaraṃ viśvataḥ paribhūrasi |\n sa iddeveṣu ghachati ||\n aghnirhotā kavikratuḥ satyaścitraśravastamaḥ |\n devo devebhirā ghamat ||\n yadaṅgha dāśuṣe tvamag

[{'url': 'https://www.sacred-texts.com/hin/rvsan/rvi01.htm',
  'book': 1,
  'hymns': [{'devanagari': 'अग्निमीळे पुरोहितं यज्ञस्य देवं रत्वीजम |\n होतारं रत्नधातमम ||\n अग्निः पूर्वेभिर्र्षिभिरीड्यो नूतनैरुत |\n स देवानेह वक्षति ||\n अग्निना रयिमश्नवत पोषमेव दिवे-दिवे |\n यशसं वीरवत्तमम ||\n अग्ने यं यज्ञमध्वरं विश्वतः परिभूरसि |\n स इद्देवेषु गछति ||\n अग्निर्होता कविक्रतुः सत्यश्चित्रश्रवस्तमः |\n देवो देवेभिरा गमत ||\n यदङग दाशुषे तवमग्ने भद्रं करिष्यसि |\n तवेत तत सत्यमङगिरः ||\n उप तवाग्ने दिवे-दिवे दोषावस्तर्धिया वयम |\n नमो भरन्त एमसि ||\n राजन्तमध्वराणां गोपां रतस्य दीदिविम |\n वर्धमानंस्वे दमे ||\n स नः पितेव सूनवे.अग्ने सूपायनो भव |\n सचस्वा नः सवस्तये ||', 'iast': '<P></P>\n aghnimīḷe purohitaṃ yajñasya devaṃ ṛtvījam |\n hotāraṃ ratnadhātamam ||\n aghniḥ pūrvebhirṛṣibhirīḍyo nūtanairuta |\n sa devāneha vakṣati ||\n aghninā rayimaśnavat poṣameva dive-dive |\n yaśasaṃ vīravattamam ||\n aghne yaṃ yajñamadhvaraṃ viśvataḥ paribhūrasi |\n sa iddeveṣu ghachati ||\n aghnirhotā kavikr